In [1]:
import cv2
import tensorflow as tf
import argparse
import numpy as np
import os
import pdb
import time
import matplotlib.pyplot as plt
import sys

/home/user/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
############### Global Parameters ###############
# path
train_path = '/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/training/'
test_path = '/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/testing/'
# demo_path = './dataset/features/testing/'
# default_model_path = './model/demo_model'
save_path = './model2/'
# video_path = './dataset/videos/training/positive/'
# batch_number
train_num = 101
test_num = 20

In [3]:
learning_rate = 0.0001
n_epochs = 30
batch_size = 6
display_step = 10

# Network Parameters
n_input = 4096 # fc6 or fc7(1*4096)
n_detection = 20 # number of object of each image (include image features)
n_hidden = 512 # hidden layer num of LSTM
n_img_hidden = 256 # embedding image features 
n_att_hidden = 256 # embedding object features
n_classes = 2 # has accident or not
n_frames = 100 # number of frame in each video 
##################################################

In [4]:

# # tf Graph input
# x = tf.placeholder("float", [None, n_frames ,n_detection, n_input])
# y = tf.placeholder("float", [None, n_classes])
# keep = tf.placeholder("float",[None])

# # Define weights
# weights = {
#     'em_obj': tf.Variable(tf.random_normal([n_input,n_att_hidden], mean=0.0, stddev=0.01)),
#     'em_img': tf.Variable(tf.random_normal([n_input,n_img_hidden], mean=0.0, stddev=0.01)),
#     'att_w': tf.Variable(tf.random_normal([n_att_hidden, 1], mean=0.0, stddev=0.01)),
#     'att_wa': tf.Variable(tf.random_normal([n_hidden, n_att_hidden], mean=0.0, stddev=0.01)),
#     'att_ua': tf.Variable(tf.random_normal([n_att_hidden, n_att_hidden], mean=0.0, stddev=0.01)),
#     'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01))
# }
# biases = {
#     'em_obj': tf.Variable(tf.random_normal([n_att_hidden], mean=0.0, stddev=0.01)),
#     'em_img': tf.Variable(tf.random_normal([n_img_hidden], mean=0.0, stddev=0.01)),
#     'att_ba': tf.Variable(tf.zeros([n_att_hidden])),
#     'out': tf.Variable(tf.random_normal([n_classes], mean=0.0, stddev=0.01))
# }

# # Define a lstm cell with tensorflow
# lstm_cell = tf.nn.rnn_cell.LSTMCell(n_hidden,initializer= tf.random_normal_initializer(mean=0.0,stddev=0.01),use_peepholes = True,state_is_tuple = False)
# # using dropout in output of LSTM
# lstm_cell_dropout = tf.nn.rnn_cell.DropoutWrapper(lstm_cell,output_keep_prob=1 - keep[0])
# # init LSTM parameters
# istate = tf.zeros([batch_size, lstm_cell.state_size])
# h_prev = tf.zeros([batch_size, n_hidden])
# # init loss 
# loss = 0.0  
# # Mask 
# zeros_object = tf.to_float(tf.not_equal(tf.reduce_sum(tf.transpose(x[:,:,1:n_detection,:],[1,2,0,3]),3),0)) # frame x n x b
# # Start creat graph
# for i in range(n_frames):
#     # input features (Faster-RCNN fc7)
#     X = tf.transpose(x[:,i,:,:], [1, 0, 2])  # permute n_steps and batch_size (n x b x h)
#     # frame embedded
#     image = tf.matmul(X[0,:,:],weights['em_img']) + biases['em_img'] # 1 x b x h
#     # object embedded
#     n_object = tf.reshape(X[1:n_detection,:,:], [-1, n_input]) # (n_steps*batch_size, n_input)
#     n_object = tf.matmul(n_object, weights['em_obj']) + biases['em_obj'] # (n x b) x h
#     n_object = tf.reshape(n_object,[n_detection-1,batch_size,n_att_hidden]) # n-1 x b x h
#     n_object = tf.multiply(n_object,tf.expand_dims(zeros_object[i],2))

#     # object attention
#     brcst_w = tf.tile(tf.expand_dims(weights['att_w'], 0), [n_detection-1,1,1]) # n x h x 1
#     image_part = tf.matmul(n_object, tf.tile(tf.expand_dims(weights['att_ua'], 0), [n_detection-1,1,1])) + biases['att_ba'] # n x b x h
#     e = tf.tanh(tf.matmul(h_prev,weights['att_wa'])+image_part) # n x b x h
#     # the probability of each object
#     alphas = tf.multiply(tf.nn.softmax(tf.reduce_sum(tf.matmul(e,brcst_w),2),0),zeros_object[i])
#     # weighting sum
#     attention_list = tf.multiply(tf.expand_dims(alphas,2),n_object)
#     attention = tf.reduce_sum(attention_list,0) # b x h
#     # concat frame & object
#     fusion = tf.concat([image,attention],1)
#     # reuse variables
#     if i > 0 :  tf.get_variable_scope().reuse_variables()       
#     with tf.variable_scope("LSTM") as vs:
#         outputs,istate = lstm_cell_dropout(fusion,istate)
#         lstm_variables = [v for v in tf.global_variables() if v.name.startswith(vs.name)]
#     # save prev hidden state of LSTM
#     h_prev = outputs
#     # FC to output
#     pred = tf.matmul(outputs,weights['out']) + biases['out'] # b x n_classes
#     # save the predict of each time step
#     if i == 0:
#         soft_pred = tf.reshape(tf.gather(tf.transpose(tf.nn.softmax(pred),(1,0)),1),(batch_size,1))
#         all_alphas = tf.expand_dims(alphas,0)
#     else:
#         temp_soft_pred = tf.reshape(tf.gather(tf.transpose(tf.nn.softmax(pred),(1,0)),1),(batch_size,1))
#         soft_pred = tf.concat([soft_pred,temp_soft_pred],1)
#         temp_alphas = tf.expand_dims(alphas,0)
#         all_alphas = tf.concat([all_alphas, temp_alphas],0)

#     # positive example (exp_loss)
#     pos_loss = -tf.multiply(tf.exp(-(n_frames-i-1)/20.0),-tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
#     # negative example
#     neg_loss = tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y) # Softmax loss

#     temp_loss = tf.reduce_mean(tf.add(tf.multiply(pos_loss,y[:,1]),tf.multiply(neg_loss,y[:,0])))
#     #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
#     loss = tf.add(loss, temp_loss)

# # Define loss and optimizer
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames) # Adam Optimizer

In [5]:
def build_model():

    # tf Graph input
    x = tf.placeholder("float", [None, n_frames ,n_detection, n_input])
    y = tf.placeholder("float", [None, n_classes])
    keep = tf.placeholder("float",[None])

    # Define weights
    weights = {
        'em_obj': tf.Variable(tf.random_normal([n_input,n_att_hidden], mean=0.0, stddev=0.01)),
        'em_img': tf.Variable(tf.random_normal([n_input,n_img_hidden], mean=0.0, stddev=0.01)),
        'att_w': tf.Variable(tf.random_normal([n_att_hidden, 1], mean=0.0, stddev=0.01)),
        'att_wa': tf.Variable(tf.random_normal([n_hidden, n_att_hidden], mean=0.0, stddev=0.01)),
        'att_ua': tf.Variable(tf.random_normal([n_att_hidden, n_att_hidden], mean=0.0, stddev=0.01)),
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01))
    }
    biases = {
        'em_obj': tf.Variable(tf.random_normal([n_att_hidden], mean=0.0, stddev=0.01)),
        'em_img': tf.Variable(tf.random_normal([n_img_hidden], mean=0.0, stddev=0.01)),
        'att_ba': tf.Variable(tf.zeros([n_att_hidden])),
        'out': tf.Variable(tf.random_normal([n_classes], mean=0.0, stddev=0.01))
    }

    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.LSTMCell(n_hidden,initializer= tf.random_normal_initializer(mean=0.0,stddev=0.01),use_peepholes = True,state_is_tuple = False)
    # using dropout in output of LSTM
    lstm_cell_dropout = tf.nn.rnn_cell.DropoutWrapper(lstm_cell,output_keep_prob=1 - keep[0])
    # init LSTM parameters
    istate = tf.zeros([batch_size, lstm_cell.state_size])
    h_prev = tf.zeros([batch_size, n_hidden])
    # init loss 
    loss = 0.0  
    # Mask 
    zeros_object = tf.to_float(tf.not_equal(tf.reduce_sum(tf.transpose(x[:,:,1:n_detection,:],[1,2,0,3]),3),0)) # frame x n x b
    # Start creat graph
    for i in range(n_frames):
        # input features (Faster-RCNN fc7)
        X = tf.transpose(x[:,i,:,:], [1, 0, 2])  # permute n_steps and batch_size (n x b x h)
        # frame embedded
        image = tf.matmul(X[0,:,:],weights['em_img']) + biases['em_img'] # 1 x b x h
        # object embedded
        n_object = tf.reshape(X[1:n_detection,:,:], [-1, n_input]) # (n_steps*batch_size, n_input)
        n_object = tf.matmul(n_object, weights['em_obj']) + biases['em_obj'] # (n x b) x h
        n_object = tf.reshape(n_object,[n_detection-1,batch_size,n_att_hidden]) # n-1 x b x h
        n_object = tf.multiply(n_object,tf.expand_dims(zeros_object[i],2))

        # object attention
        brcst_w = tf.tile(tf.expand_dims(weights['att_w'], 0), [n_detection-1,1,1]) # n x h x 1
        image_part = tf.matmul(n_object, tf.tile(tf.expand_dims(weights['att_ua'], 0), [n_detection-1,1,1])) + biases['att_ba'] # n x b x h
        e = tf.tanh(tf.matmul(h_prev,weights['att_wa'])+image_part) # n x b x h
        # the probability of each object
        alphas = tf.multiply(tf.nn.softmax(tf.reduce_sum(tf.matmul(e,brcst_w),2),0),zeros_object[i])
        # weighting sum
        attention_list = tf.multiply(tf.expand_dims(alphas,2),n_object)
        attention = tf.reduce_sum(attention_list,0) # b x h
        # concat frame & object
        fusion = tf.concat([image,attention],1)
        # reuse variables
        if i > 0 :  tf.get_variable_scope().reuse_variables()       
        with tf.variable_scope("LSTM") as vs:
            outputs,istate = lstm_cell_dropout(fusion,istate)
            lstm_variables = [v for v in tf.global_variables() if v.name.startswith(vs.name)]
        # save prev hidden state of LSTM
        h_prev = outputs
        # FC to output
        pred = tf.matmul(outputs,weights['out']) + biases['out'] # b x n_classes
        # save the predict of each time step
        if i == 0:
            soft_pred = tf.reshape(tf.gather(tf.transpose(tf.nn.softmax(pred),(1,0)),1),(batch_size,1))
            all_alphas = tf.expand_dims(alphas,0)
        else:
            temp_soft_pred = tf.reshape(tf.gather(tf.transpose(tf.nn.softmax(pred),(1,0)),1),(batch_size,1))
            soft_pred = tf.concat([soft_pred,temp_soft_pred],1)
            temp_alphas = tf.expand_dims(alphas,0)
            all_alphas = tf.concat([all_alphas, temp_alphas],0)

        # positive example (exp_loss)
        pos_loss = -tf.multiply(tf.exp(-(n_frames-i-1)/20.0),-tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
        # negative example
        neg_loss = tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y) # Softmax loss

        temp_loss = tf.reduce_mean(tf.add(tf.multiply(pos_loss,y[:,1]),tf.multiply(neg_loss,y[:,0])))
        #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
        loss = tf.add(loss, temp_loss)
        
    # Define loss and optimizer
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames) # Adam Optimizer

    return x,keep,y,optimizer,loss,lstm_variables,soft_pred,all_alphas


In [6]:
annotation_file = '/media/user/Hard_Disk/Dataset/child_accident_2/annotation/accident_frame.txt'
w = open(annotation_file, "r")
ann = w.read()
annotation_data = []
for i in ann.split("\n"):
    b = i.split(" ")
    if (len(b) > 1):
        annotation_data.append(b[1])
annotation_data = np.array(annotation_data).astype("int32")

In [7]:
def build_data(data):
    batch_image_x = []
    for i in range(6):
        if (data['label'][i][0] == 1):
#             batch_action_x.append(data['action'][i][50:150])
            batch_image_x.append(data['data'][i][50:150])

        else:
            start = annotation_data[int(data['id'][i])-1]
#             batch_action_x.append(data['action'][i][start-90:start+10])
            batch_image_x.append(data['data'][i][start-90:start+10])

    return np.array(batch_image_x)

In [8]:
def test_all(sess,num,path,x,keep,y,loss,lstm_variables,soft_pred):
    total_loss = 0.0

    for num_batch in range(1,num+1):
        # load test_data
        file_name = '%03d' %num_batch
        test_all_data = np.load(path+'batch_'+file_name+'.npz')
        test_data = build_data(test_all_data)
        test_labels = test_all_data['label']
        [temp_loss,pred] = sess.run([loss,soft_pred], feed_dict={x: test_data, y: test_labels, keep: [0.0]})
         
        total_loss += temp_loss/batch_size

        if num_batch <= 1:
            all_pred = pred[:,0:90]
            all_labels = np.reshape(test_labels[:,1],[batch_size,1])
        else:
            all_pred = np.vstack((all_pred,pred[:,0:90]))
            all_labels = np.vstack((all_labels,np.reshape(test_labels[:,1],[batch_size,1])))

    evaluation(all_pred,all_labels)

In [9]:
def train():
    # build model
    x,keep,y,optimizer,loss,lstm_variables,soft_pred,all_alphas = build_model()
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.95)
    sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
    # mkdir folder for saving model
    if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
    # Initializing the variables
    init = tf.global_variables_initializer()
    # Launch the graph
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=100)
    # Keep training until reach max iterations
    # start training
    for epoch in range(n_epochs):
        # random chose batch.npz
        epoch_loss = np.zeros((train_num,1),dtype = float)
        n_batchs = np.arange(1,train_num+1)
        np.random.shuffle(n_batchs)
        tStart_epoch = time.time()
        for batch in n_batchs:
            file_name = '%03d' %batch
            batch_data = np.load(train_path+'batch_'+file_name+'.npz')
            batch_xs = build_data(batch_data)
            
            batch_ys = batch_data['label']
            print(batch_xs.shape, batch_ys.shape)
            _,batch_loss = sess.run([optimizer,loss], feed_dict={x: batch_xs, y: batch_ys, keep: [0.5]})
            epoch_loss[batch-1] = batch_loss/batch_size
        # print one epoch
        print ("Epoch:", epoch+1, " done. Loss:", np.mean(epoch_loss))
        tStop_epoch = time.time()
        print ("Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s")
        sys.stdout.flush()
        if (epoch+1) %5 == 0:
            saver.save(sess,save_path+"model", global_step = epoch+1)
            print ("Training")
            test_all(sess,train_num,train_path,x,keep,y,loss,lstm_variables,soft_pred)
            print ("Testing")
            test_all(sess,test_num,test_path,x,keep,y,loss,lstm_variables,soft_pred)
    print ("Optimization Finished!")
    saver.save(sess, save_path+"final_model")

In [10]:
def evaluation(all_pred,all_labels, total_time = 90, vis = False, length = None):
    ### input: all_pred (N x total_time) , all_label (N,)
    ### where N = number of videos, fps = 20 , time of accident = total_time
    ### output: AP & Time to Accident

    if length is not None:
        all_pred_tmp = np.zeros(all_pred.shape)
        for idx, vid in enumerate(length):
                all_pred_tmp[idx,total_time-vid:] = all_pred[idx,total_time-vid:]
        all_pred = np.array(all_pred_tmp)
        temp_shape = sum(length)
    else:
        length = [total_time] * all_pred.shape[0]
        temp_shape = all_pred.shape[0]*total_time
    Precision = np.zeros((temp_shape))
    Recall = np.zeros((temp_shape))
    Time = np.zeros((temp_shape))
    cnt = 0
    AP = 0.0
    for Th in sorted(all_pred.flatten()):
        if length is not None and Th == 0:
                continue
        Tp = 0.0
        Tp_Fp = 0.0
        Tp_Tn = 0.0
        time = 0.0
        counter = 0.0
        for i in range(len(all_pred)):
            tp =  np.where(all_pred[i]*all_labels[i]>=Th)
            Tp += float(len(tp[0])>0)
            if float(len(tp[0])>0) > 0:
                time += tp[0][0] / float(length[i])
                counter = counter+1
            Tp_Fp += float(len(np.where(all_pred[i]>=Th)[0])>0)
        if Tp_Fp == 0:
            Precision[cnt] = np.nan
        else:
            Precision[cnt] = Tp/Tp_Fp
        if np.sum(all_labels) ==0:
            Recall[cnt] = np.nan
        else:
            Recall[cnt] = Tp/np.sum(all_labels)
        if counter == 0:
            Time[cnt] = np.nan
        else:
            Time[cnt] = (1-time/counter)
        cnt += 1

    new_index = np.argsort(Recall)
    Precision = Precision[new_index]
    Recall = Recall[new_index]
    Time = Time[new_index]
    _,rep_index = np.unique(Recall,return_index=1)
    new_Time = np.zeros(len(rep_index))
    new_Precision = np.zeros(len(rep_index))
    for i in range(len(rep_index)-1):
        new_Time[i] = np.max(Time[rep_index[i]:rep_index[i+1]])
        new_Precision[i] = np.max(Precision[rep_index[i]:rep_index[i+1]])

    new_Time[-1] = Time[rep_index[-1]]
    new_Precision[-1] = Precision[rep_index[-1]]
    new_Recall = Recall[rep_index]
    new_Time = new_Time[~np.isnan(new_Precision)]
    new_Recall = new_Recall[~np.isnan(new_Precision)]
    new_Precision = new_Precision[~np.isnan(new_Precision)]

    if new_Recall[0] != 0:
        AP += new_Precision[0]*(new_Recall[0]-0)
    for i in range(1,len(new_Precision)):
        AP += (new_Precision[i-1]+new_Precision[i])*(new_Recall[i]-new_Recall[i-1])/2

    print ("Average Precision= " + "{:.4f}".format(AP) + " ,mean Time to accident= " +"{:.4}".format(np.mean(new_Time) * 5))
    sort_time = new_Time[np.argsort(new_Recall)]
    sort_recall = np.sort(new_Recall)
    print ("Recall@80%, Time to accident= " +"{:.4}".format(sort_time[np.argmin(np.abs(sort_recall-0.8))] * 5))

    ### visualize



In [11]:
if __name__ == '__main__':
    train()
#     if args.mode == 'train':
#            train()
#     elif args.mode == 'test':
#            test(args.model)
#     elif args.mode == 'demo':
#            vis(args.model)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6

(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
Epoch: 3  done. Loss: 4.176527599296947
Epoch Time Cost: 825.2 s
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)

(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
Epoch: 6  done. Loss: 3.754654711229179
Epoch Time Cost: 905.39 s
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2

(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
Epoch: 9  done. Loss: 3.2823722181540513
Epoch Time Cost: 889.77 s
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 

(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
Epoch: 12  done. Loss: 3.0462958033722227
Epoch Time Cost: 908.6 s
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 2)
(6, 100, 20, 4096) (6, 

KeyboardInterrupt: 